In [ ]:
pip install imblearn

     |████████████████████████████████| 199 kB 11.1 MB/s            
     |████████████████████████████████| 189 kB 70.6 MB/s            


In [2]:
import boto3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, classification_report
from imblearn.over_sampling import SMOTE


# Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'gabriel-predictive-analytics'
bucket = s3.Bucket(bucket_name)

# Defining the file to be read from s3 bucket
file_key = "telecom_churn.csv"

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# Reading the csv file
churn = pd.read_csv(file_content_stream)
churn.head(1)

,Churn,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,0,128,1,1,2.7,1,265.1,110,89.0,9.87,10.0


In [3]:
# Relative Frequency table
churn['Churn'].value_counts() / churn.shape[0]

0    0.855086
1    0.144914
Name: Churn, dtype: float64

In [4]:
# Defining the input and target variables
X = churn[['AccountWeeks', 'ContractRenewal', 'CustServCalls', 'MonthlyCharge', 'DayMins']]
Y = churn['Churn']

# Splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y)

In [6]:
# Running SMOTE
X_SMOTE, Y_SMOTE = SMOTE().fit_resample(X_train, Y_train)

In [7]:
Y_SMOTE.value_counts()

1    2280
0    2280
Name: Churn, dtype: int64

In [9]:
# Random Forest Classifier model
RF_md = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_SMOTE, Y_SMOTE)

# Predicting on test dataset
RF_preds = RF_md.predict_proba(X_test)[:,1]

# Computing the ROC curve
fpr, tpr, threshold = roc_curve(Y_test, RF_preds)

# Creating a data-frame
cutoff_values = pd.DataFrame({'False_Positive': fpr, 'True_Positive': tpr, 'Cutoff': threshold})

# fiding the cutoff value close to the perfect model (tpr = 1, fpr = 0)
cutoff_values['True_Positive_minus_1'] = cutoff_values['True_Positive'] -1
cutoff_values['Distance_to_perfect_model'] = np.sqrt(cutoff_values['False_Positive']**2 + cutoff_values['True_Positive_minus_1']**2)
cutoff_values = cutoff_values.sort_values(by = 'Distance_to_perfect_model').reset_index(drop = True)

## Changing likelihoods to labels
RF_preds = np.where(RF_preds < cutoff_values['Cutoff'][0], 0, 1)

# Printing classification report
print(classification_report(Y_test, RF_preds))

              precision    recall  f1-score   support

           0       0.97      0.86      0.91       570
           1       0.51      0.86      0.64        97

    accuracy                           0.86       667
   macro avg       0.74      0.86      0.77       667
weighted avg       0.90      0.86      0.87       667



In [10]:
# Adaboost Classifier Model
ADA_md = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500, learning_rate = 0.01).fit(X_SMOTE, Y_SMOTE)

# Predicting on Test dataaset
ADA_preds = ADA_md.predict_proba(X_test)[:,1]

# Computing the ROC curve
fpr, tpr, threshold = roc_curve(Y_test, ADA_preds) 

cutoff_values = pd.DataFrame({"False_Positive": fpr, "True_Positive": tpr, "Cutoff": threshold})

# fiding the cutoff value close to the perfect model (tpr = 1, fpr = 0)
cutoff_values['True_Positive_minus_1'] = cutoff_values['True_Positive'] -1
cutoff_values['Distance_to_perfect_model'] = np.sqrt(cutoff_values['False_Positive']**2 + cutoff_values['True_Positive_minus_1']**2)
cutoff_values = cutoff_values.sort_values(by = 'Distance_to_perfect_model').reset_index(drop = True)

## Changing likelihoods to labels
ADA_preds = np.where(ADA_preds < cutoff_values['Cutoff'][0], 0, 1)

# Printing classification report
print(classification_report(Y_test, ADA_preds))

              precision    recall  f1-score   support

           0       0.98      0.87      0.92       570
           1       0.54      0.88      0.67        97

    accuracy                           0.87       667
   macro avg       0.76      0.87      0.79       667
weighted avg       0.91      0.87      0.88       667



### Based on my results, I would use AdaBoostClassifier to predict class 1

In [15]:
cutoff_values['Cutoff'][0]

0.4654803485040742

In [12]:
cutoff_values.head()

,False_Positive,True_Positive,Cutoff,True_Positive_minus_1,Distance_to_perfect_model
0,0.128070,0.876289,0.465480,-0.123711,0.178063
1,0.122807,0.865979,0.469048,-0.134021,0.181778
2,0.128070,0.865979,0.466674,-0.134021,0.185374
3,0.108772,0.845361,0.487139,-0.154639,0.189062
4,0.152632,0.876289,0.458017,-0.123711,0.196471
